__Accomplishments__

____________________________________
Accompli :
- Keep certifs
- Projets => Nombres de projets 
- +/- Verify skills
- Honors keep ( NLP +/- ou nombre )
- Keep languages
- Organizations => Nombre of organizations
____________________________________

In [13]:
import datetime
from dateutil import relativedelta
import numpy as np

In [1]:
import pandas as pd
df = pd.read_pickle("dummy.pkl")
df = df.drop(columns=["scraped_time"])
df = df.drop(columns=["interests"])
df = df.drop(columns=["search"])

In [2]:
json_array= []

for i in range(len(df)):
    json_array.append(df["accomplishments"][i])

store_list = []

for item in json_array:
    store_details = {"courses":None,"certifications":None, "projects":None,"languages":None,"nbre_languages":None,"honors":None,"organizations":None,"nbre_projet":None}
    store_details['certifications'] = len(item['certifications'])
    store_details['courses'] = len(item['courses'])
    store_details['projects'] = item['projects']
    store_details['languages'] = item['languages']
    store_details['honors'] = len(item['honors'])
    store_details['organizations'] = len(item['organizations'])
    store_details['nbre_projet'] = len(item['projects'])
    store_details['nbre_languages'] = len(item['languages'])
    store_list.append(store_details)

dtv=pd.DataFrame(store_list,index=None)
dtv=dtv.drop(columns="courses")
dtv=dtv.drop(columns="projects")
dtv=dtv.drop(columns="languages")
df=df.drop(columns="accomplishments")


In [3]:
df = pd.concat([df, dtv], axis=1, sort=False)

In [4]:
df["url"] = "*****************"
df["personal_info"] = "*****************"
df

,url,personal_info,skills,experiences,certifications,nbre_languages,honors,organizations,nbre_projet
0,*****************,*****************,"[{'name': 'AutoCAD', 'endorsements': '5'}, {'n...","{'jobs': [{'title': 'Architecte Principal', 'c...",0,0,0,0,0
1,*****************,*****************,"[{'name': 'Microsoft Office', 'endorsements': ...","{'jobs': [{'title': 'Founder and CEO', 'compan...",0,3,0,0,5
2,*****************,*****************,"[{'name': 'Microsoft Office', 'endorsements': ...","{'jobs': [{'title': 'Projet de fin d’études', ...",0,4,0,0,0
3,*****************,*****************,"[{'name': 'Java', 'endorsements': '6'}, {'name...","{'jobs': [{'title': 'Technical Lead', 'company...",0,1,0,0,4
4,*****************,*****************,"[{'name': 'R', 'endorsements': '7'}, {'name': ...",{'jobs': [{'title': 'PhD. Degree on Statistics...,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
9995,*****************,*****************,"[{'name': 'Python', 'endorsements': '37'}, {'n...",{'jobs': [{'title': 'Développeur Python/Devops...,0,2,0,0,0
9996,*****************,*****************,"[{'name': 'Management', 'endorsements': '1'}, ...",{'jobs': [{'title': 'Responsable Financement d...,0,0,0,0,0
9997,*****************,*****************,"[{'name': 'Java Enterprise Edition', 'endorsem...","{'jobs': [{'title': 'SAP Software Engineer', '...",0,0,0,0,0
9998,*****************,*****************,[],"{'jobs': [{'title': 'Médecin', 'company': 'Cna...",0,0,0,0,0


## Integration of the Feature Tenure
- Calcultating the probability for a person to stay working in the company during a minimum period mentioned by the recruiter

In [56]:
"""
Description:
    a dict shape which contains, as keys, the labels of the months as used in the date shape in LinkedIn, and their corresponding integer value [1 .. 12]
"""
date_dict = {'Jan' : 1,
            'Feb' : 2,
            'Mar' : 3,
            'Apr' : 4,
            'May' : 5,
            'Jun' : 6,
            'Jul' : 7,
            'Aug' : 8,
            'Sep' : 9,
            'Oct' : 10,
            'Nov' : 11,
            'Dec' : 12,
            'juin' : 6,
            'sept.' : 9,
            'mai' : 5,
            'mars' : 3,
            'févr.' : 2,
            'avr.' : 4,
            'juil.' : 7,
            'oct.' : 10,
            'août' : 8,
            'janv.' : 1,
            'nov.' : 11,
             'déc.' : 12
            }

In [55]:
def Tenure_Ratio_Transformation(ratio, x):
    """
    Description:
        returns the ratio value if x (an integer) is greater or equal to the ratio, and it returns the value of x if x is lower than the ratio
    parameters:
        ratio: an integer to compare with
        x: an integer. The target to compare and change its value
    returns:
        an integer: the result of the operation
    """
    if x >= ratio:
        return ratio
    else:
        return x

In [54]:
def is_Present(date_range):
    """
    Description:
        Checks if the candidate in currently working at the job
    parameters:
        date_range: the date range of the experience
    returns:
        True if present, False if not
    """
    if "Aujourd’hui" in date_range or "Present" in date_range:
        return True
    return False

In [50]:
def Calculate_Year_Month(date_range):
    """
    Description:
        Calculates the duration in months of a given date range
    parameters:
        date_range: shape of ['start month', 'start year', 'end month', 'end year']
    returns:
        duration of an experience in months
    """
    start_date=datetime.datetime(date_range[1],date_range[0],1)
    end_date=datetime.datetime(date_range[3],date_range[2],1)
    r = relativedelta.relativedelta(end_date, start_date)
    years=r.years
    months=r.months
    return (months + years*12)

In [51]:
def Calculate_Duration(person):
    """
    Description:
        Calculates the Duration of all the experiences of a person passed in parameter
    parameter: 
        person: a person's all experiences
    returns:
        present_durations: array shape of the duration of each present jobs
        past_duration: array shape of the duration of each past jobs
    """
    present_durations = []
    past_duration = []
    nb_trainings = 0
    nbr_present_jobs = 0
    experience_duration = 0
    for experience in person["jobs"]:
        if any(c in experience['title'] for c in ['stagiaire', 'Stagiaire', 'stage', 'Stage', 'STAGE', 'Trainee', 'trainee', 'Internship', 'internship']):
            nb_trainings = nb_trainings+1
        else:
            date_range = experience["date_range"]
            if date_range is None:
                return present_durations, past_duration
            date_range_liste = date_range.split()
            if len(date_range_liste) == 3:
                if is_Present(date_range_liste):
                    nbr_present_jobs += 1
                    if nbr_present_jobs <= 1 :
                        experience_duration = Calculate_Year_Month([1,int(date_range_liste[0]), 2, 2020])
                        present_durations.append(experience_duration)
                        previous_year = int(date_range_liste[0])
                        previous_month = 1
                    else:
                        experience_duration = Calculate_Year_Month([1,int(date_range_liste[0]), previous_month, previous_year])
                        past_duration.append(experience_duration)
                        previous_year = int(date_range_liste[0])
                        previous_month = 1
                else:
                    experience_duration = Calculate_Year_Month([1,int(date_range_liste[0]), 2, int(date_range_liste[2])])
                    past_duration.append(experience_duration)
                    previous_year = int(date_range_liste[0])
                    previous_month = 1
            elif len(date_range_liste) >= 4:
                if is_Present(date_range_liste):
                    nbr_present_jobs += 1
                    if nbr_present_jobs <= 1:
                        if date_range_liste[0] in date_dict.keys():
                            experience_duration = Calculate_Year_Month([date_dict[date_range_liste[0]],int(date_range_liste[1]), 2, 2020])
                        else:
                            experience_duration = Calculate_Year_Month([1,int(date_range_liste[0]), 2, 2020])
                        present_durations.append(experience_duration)
                        previous_year = int(date_range_liste[1])
                        previous_month = date_dict[date_range_liste[0]]
                    else:
                        if date_range_liste[0] in date_dict.keys():
                            experience_duration = Calculate_Year_Month([date_dict[date_range_liste[0]],int(date_range_liste[1]), previous_month, previous_year])
                        else:
                            experience_duration = Calculate_Year_Month([1,int(date_range_liste[0]), previous_month, previous_year])
                        past_duration.append(experience_duration)
                        previous_year = int(date_range_liste[1])
                        previous_month = date_dict[date_range_liste[0]]                      
                else:
                    if date_range_liste[0] in date_dict.keys():
                        if date_range_liste[3] in date_dict.keys():
                            experience_duration = Calculate_Year_Month([date_dict[date_range_liste[0]],int(date_range_liste[1]), date_dict[date_range_liste[3]], int(date_range_liste[4])])
                        else:
                            experience_duration = Calculate_Year_Month([date_dict[date_range_liste[0]],int(date_range_liste[1]), 1, int(date_range_liste[3])])

                        previous_year = int(date_range_liste[1])
                        previous_month = date_dict[date_range_liste[0]]
                    else:
                        experience_duration = Calculate_Year_Month([1,int(date_range_liste[0]), date_dict[date_range_liste[2]], int(date_range_liste[3])])
                        previous_year = int(date_range_liste[0])
                        previous_month = 1
                    past_duration.append(experience_duration)
                    
    return present_durations, past_duration

In [52]:
def calculate_Tenure(present_durations, past_duration, ratio):
    """
    Description:
        Calculates the probability of a candidate to stay at a job post for a minimum period passed in parameter (the ratio)
    parameters:
        present_durations: array shape of the duration of each present jobs
        past_duration: array shape of the duration of each past jobs
        ratio: the requested minimum duration in months
    returns:
        the Tenure in percentage [0, 1]
    """
    past_duration_sigmoid = []
    present_durations_sigmoid = []
    for d in past_duration:
        past_duration_sigmoid.append(Tenure_Ratio_Transformation(ratio, d))
    for d in present_durations:
        present_durations_sigmoid.append(Tenure_Ratio_Transformation(ratio, d))
    
    if len(present_durations_sigmoid) == 0:
        present_durations_sigmoid.append(0)
    if len(past_duration_sigmoid) == 0:
        past_duration_sigmoid.append(0)
    present_array = np.array(present_durations_sigmoid)
    past_array =  np.array(past_duration_sigmoid)
    
    return round(((0.7 * sum(past_array / ratio) / len(past_array)) + (0.3 * (present_array / ratio)))[0],2)

In [42]:
df_tenure = pd.DataFrame(columns=['Tenure']) #Creating an empty pandas dataframe for the tenure, this will be later concatenated with the intial dataframe

## Here we are going to calculate the tenure for 3 years (36 months) 

In [44]:
i=0
for person in df.experiences:
    pres, past = Calculate_Duration(person)
    df_tenure = df_tenure.append([{'Tenure' : calculate_Tenure(pres,past, 36)}],ignore_index=True)

In [45]:
df_tenure

,Tenure
0,0.92
1,0.33
2,0.08
3,0.32
4,0.33
...,...
9995,0.38
9996,1.00
9997,0.10
9998,0.00


In [46]:
df_final = pd.concat([df,df_tenure], axis=1)

## The result of the new Dataset 

In [47]:
df_final

,url,personal_info,skills,experiences,certifications,nbre_languages,honors,organizations,nbre_projet,Tenure
0,*****************,*****************,"[{'name': 'AutoCAD', 'endorsements': '5'}, {'n...","{'jobs': [{'title': 'Architecte Principal', 'c...",0,0,0,0,0,0.92
1,*****************,*****************,"[{'name': 'Microsoft Office', 'endorsements': ...","{'jobs': [{'title': 'Founder and CEO', 'compan...",0,3,0,0,5,0.33
2,*****************,*****************,"[{'name': 'Microsoft Office', 'endorsements': ...","{'jobs': [{'title': 'Projet de fin d’études', ...",0,4,0,0,0,0.08
3,*****************,*****************,"[{'name': 'Java', 'endorsements': '6'}, {'name...","{'jobs': [{'title': 'Technical Lead', 'company...",0,1,0,0,4,0.32
4,*****************,*****************,"[{'name': 'R', 'endorsements': '7'}, {'name': ...",{'jobs': [{'title': 'PhD. Degree on Statistics...,0,0,0,0,0,0.33
...,...,...,...,...,...,...,...,...,...,...
9995,*****************,*****************,"[{'name': 'Python', 'endorsements': '37'}, {'n...",{'jobs': [{'title': 'Développeur Python/Devops...,0,2,0,0,0,0.38
9996,*****************,*****************,"[{'name': 'Management', 'endorsements': '1'}, ...",{'jobs': [{'title': 'Responsable Financement d...,0,0,0,0,0,1.00
9997,*****************,*****************,"[{'name': 'Java Enterprise Edition', 'endorsem...","{'jobs': [{'title': 'SAP Software Engineer', '...",0,0,0,0,0,0.10
9998,*****************,*****************,[],"{'jobs': [{'title': 'Médecin', 'company': 'Cna...",0,0,0,0,0,0.00
